# Example code for loading contact mapping data from PICO-db

- cell 1:
    - load the contact mapping file into a dictionary of human and corresponding object part keys
    - body parts not mapped to object contact are removed
    - convert the human vertices into SMPL-X from SMPL
- cell 2:
    - at this point, the human contact data is a list of vertices
    - and the object contact data is barycentric coordinates of points on the faces
    - we can convert these to 3d coordinates for the sake of calculations (such as contact loss between the two parts of the interaction)
- cell 3 & 4:
    - the human and object meshes are loaded
    - for the sake of demonstration, the contact points are visualized as small spheres on the human and object meshes

In [35]:
import sys, os
import trimesh
import numpy as np
import pickle
sys.path.append(os.path.abspath(".."))
from src.utils.contact_mapping import load_contact_mapping, interpret_contact_points


def convert_contact_map_to_smplx(contact_transfer_map: dict) -> dict:
    # load smplx to smpl matrix
    with open('../static/smpl_to_smplx.pkl', 'rb') as f:
        smpl_to_smplx = pickle.load(f)

    for key in contact_transfer_map:
        if key.startswith('human'):
            smplx_contact = []
            for v in contact_transfer_map[key]:
                v = int(v[1:])
                smpl_vertices = np.zeros((6890))
                smpl_vertices[v] = 1
                smplx_vertices = smpl_vertices @ smpl_to_smplx['matrix'].T
                    
                best = np.argmax(smplx_vertices)
                smplx_contact.append('v ' + str(best))

            contact_transfer_map[key] = smplx_contact

    return contact_transfer_map



contact_map = load_contact_mapping(
    contact_map_path="../demo_input/skateboard__vcoco_000000012938/corresponding_contacts.json",
    convert_to_smplx=False
)
contact_map = convert_contact_map_to_smplx(contact_map)



removed humanShapeleftfoottop from contact_transfer_map
contact_transfer_map processed:
... Shapeleftfoot: h 81, o 81


In [36]:
obj_mesh = trimesh.load("../demo_input/skateboard__vcoco_000000012938/object.obj", process=False)

human_params = np.load("../demo_input/skateboard__vcoco_000000012938/osx_human.npz", allow_pickle=True)
human_vertices = human_params['smpl_vertices'][0]

human_points, object_points = interpret_contact_points(
    contact_map,
    human_vertices,
    obj_mesh
)
print("Human contact points:", len(human_points))
print("Object contact points:", len(object_points))

Human contact points: 81
Object contact points: 81


In [37]:
proto = trimesh.creation.icosphere(radius=0.01, subdivisions=2)

object_contact_spheres = [proto.copy().apply_translation(p) for p in object_points.cpu().numpy()]
object_contact_scene = trimesh.Scene(trimesh.util.concatenate([obj_mesh, *object_contact_spheres]))
object_contact_scene.show()


In [38]:
human_faces = np.load('../static/smplx_faces.npy').astype(int)
human_mesh = trimesh.Trimesh(vertices=human_vertices, faces=human_faces, process=False)

human_contact_spheres = [proto.copy().apply_translation(p) for p in human_points.cpu().numpy()]
human_contact_scene = trimesh.Scene(trimesh.util.concatenate([human_mesh, *human_contact_spheres]))
human_contact_scene.show()
